In [1]:
import pandas as pd
import plotly.express as px

In [2]:
df = pd.read_csv('/content/drive/MyDrive/ecommerce_data.csv')

In [3]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


- 시간의 흐름에 따라 매출, 주문고객수, 주문단가의 추이
- 리텐션 : 시간의 흐름에 따라 고객들이 얼마나 남고, 이탈했는가
    - 유저가 제품을 사용한 이후 일정 기간이 지난 시점에 제품을 계속 사용하고 있는지 유저의 잔존과 이탈을 트래킹
    - day0에 앱에 방문한 유저 중 day1에 다시 재방문한 유저의 비율이 리텐션
    - 일반적으로 리텐션이 높으면 유저가 서비스를 주기적으로 사용한다는 뜻으로 해석 가능, 유저의 참여와 충성도 같은 지표를 높이기 위한 제품 방향
- rfm : recency, frequency, monetary를 기반으로 고객을 유형화하는 방법 , 맞춤형 전략 사용 가능

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


In [5]:
# 고객 아이디가 없는 행은 삭제
df.dropna(subset=['CustomerID'],inplace=True)

In [6]:
# 데이터 타입 변경
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'], format= '%m/%d/%Y %H:%M')
df['CustomerID'] = df['CustomerID'].astype(int).astype(str)

In [7]:
df['date_ymd'] = df['InvoiceDate'].dt.date.astype('datetime64[ns]')
df['year'] = df['InvoiceDate'].dt.year

In [8]:
# 매출 칼럼
df['amount'] = df['Quantity'] * df['UnitPrice']

In [9]:
df = df.query('Quantity > 0')

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 397924 entries, 0 to 541908
Data columns (total 11 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    397924 non-null  object        
 1   StockCode    397924 non-null  object        
 2   Description  397924 non-null  object        
 3   Quantity     397924 non-null  int64         
 4   InvoiceDate  397924 non-null  datetime64[ns]
 5   UnitPrice    397924 non-null  float64       
 6   CustomerID   397924 non-null  object        
 7   Country      397924 non-null  object        
 8   date_ymd     397924 non-null  datetime64[ns]
 9   year         397924 non-null  int32         
 10  amount       397924 non-null  float64       
dtypes: datetime64[ns](2), float64(2), int32(1), int64(1), object(5)
memory usage: 34.9+ MB


In [11]:
# 1.시간의 흐름에 따라 매출의 추이
amount_by_date = df.groupby('date_ymd')[['amount']].sum().reset_index()

fig = px.line(data_frame=amount_by_date, x='date_ymd', y='amount')
fig.show()

In [12]:
# 시간의 따른 주문 고객수
customer_count_by_date = df.groupby('date_ymd')[['CustomerID']].nunique().reset_index().rename({'CustomerID' : 'Customer_count'},axis=1)

fig = px.line(data_frame=customer_count_by_date, x='date_ymd', y='Customer_count')
fig.show()

In [13]:
# 주문단가
amount_by_date.head()

invoice_count_by_date = df.groupby('date_ymd')[['InvoiceNo']].nunique().reset_index().rename({'InvoiceNo' : 'Invoice_count'},axis=1)

In [14]:
invoice_amount = pd.merge(amount_by_date, invoice_count_by_date, on='date_ymd')
invoice_amount['amount_per_invoice'] = invoice_amount['amount']/ invoice_amount['Invoice_count']
invoice_amount

,date_ymd,amount,Invoice_count,amount_per_invoice
0,2010-12-01,46376.49,121,383.276777
1,2010-12-02,47316.53,137,345.376131
2,2010-12-03,23921.71,57,419.679123
3,2010-12-05,31771.60,87,365.190805
4,2010-12-06,31215.64,94,332.081277
...,...,...,...,...
300,2011-12-05,58202.21,116,501.743190
301,2011-12-06,46144.04,110,419.491273
302,2011-12-07,69354.21,104,666.867404
303,2011-12-08,50519.41,113,447.074425


In [15]:
fig = px.line(data_frame= invoice_amount, x='date_ymd', y='amount_per_invoice')
fig.show()

In [16]:
# 리텐션 분석 : 시간의 흐름에 따른 고객의 재방문, 이탈률
retention_df = df[['CustomerID','InvoiceDate','date_ymd']].drop_duplicates()
retention_df['date_ym'] = retention_df['date_ymd'].dt.to_period('M')
retention_df.head()

,CustomerID,InvoiceDate,date_ymd,date_ym
0,17850,2010-12-01 08:26:00,2010-12-01,2010-12
7,17850,2010-12-01 08:28:00,2010-12-01,2010-12
9,13047,2010-12-01 08:34:00,2010-12-01,2010-12
25,13047,2010-12-01 08:35:00,2010-12-01,2010-12
26,12583,2010-12-01 08:45:00,2010-12-01,2010-12


In [17]:
print(min(df['date_ymd'].unique()))
print(max(df['date_ymd'].unique()))

2010-12-01 00:00:00
2011-12-09 00:00:00


In [18]:
# 2011-12월의 데이터는 보지 않음
retention_df = retention_df.query('date_ymd <= "2011-11-30"')

In [19]:
# 계산
df_ym_list = sorted(list(retention_df['date_ym'].unique()))

In [20]:
period_start = df_ym_list[0] # 2010-12
period_target = df_ym_list[1] # 2011-01

period_start_users = set(retention_df.query('date_ym == @period_start')['CustomerID'])
period_target_users = set(retention_df.query('date_ym == @period_target')['CustomerID'])

In [21]:
retained_users = period_start_users.intersection(period_target_users)

In [22]:
retention_rate = len(retained_users) / len(period_start_users)

In [23]:
retention_rate

0.36610169491525424

In [24]:
from tqdm.notebook import tqdm
# 2010/12 -> 01,02,03 ....
# 2011/10 -> 2011/11
for i in df_ym_list:
  for j in df_ym_list:
    if i <= j :
      print(i,j)

2010-12 2010-12
2010-12 2011-01
2010-12 2011-02
2010-12 2011-03
2010-12 2011-04
2010-12 2011-05
2010-12 2011-06
2010-12 2011-07
2010-12 2011-08
2010-12 2011-09
2010-12 2011-10
2010-12 2011-11
2011-01 2011-01
2011-01 2011-02
2011-01 2011-03
2011-01 2011-04
2011-01 2011-05
2011-01 2011-06
2011-01 2011-07
2011-01 2011-08
2011-01 2011-09
2011-01 2011-10
2011-01 2011-11
2011-02 2011-02
2011-02 2011-03
2011-02 2011-04
2011-02 2011-05
2011-02 2011-06
2011-02 2011-07
2011-02 2011-08
2011-02 2011-09
2011-02 2011-10
2011-02 2011-11
2011-03 2011-03
2011-03 2011-04
2011-03 2011-05
2011-03 2011-06
2011-03 2011-07
2011-03 2011-08
2011-03 2011-09
2011-03 2011-10
2011-03 2011-11
2011-04 2011-04
2011-04 2011-05
2011-04 2011-06
2011-04 2011-07
2011-04 2011-08
2011-04 2011-09
2011-04 2011-10
2011-04 2011-11
2011-05 2011-05
2011-05 2011-06
2011-05 2011-07
2011-05 2011-08
2011-05 2011-09
2011-05 2011-10
2011-05 2011-11
2011-06 2011-06
2011-06 2011-07
2011-06 2011-08
2011-06 2011-09
2011-06 2011-10
2011-06 

In [25]:
retention = pd.DataFrame()
for i in tqdm(df_ym_list):
  for j in df_ym_list :
    period_start = i
    period_target = j

    if period_start <= period_target :
      period_start_users = set(retention_df.query('date_ym == @period_start')['CustomerID'])
      period_target_users = set(retention_df.query('date_ym == @period_target')['CustomerID'])

      retained_users = period_start_users.intersection(period_target_users)
      retention_rate = len(retained_users) / len(period_start_users)

      temp = pd.DataFrame({'cohort' : [period_start], 'date_ym':[period_target],'retention_rate':[retention_rate]})
      retention= pd.concat([retention,temp])


  0%|          | 0/12 [00:00<?, ?it/s]

In [26]:
retention['cohort_size_month'] = retention.apply(lambda x: (x['date_ym'] - x['cohort']).n,axis=1)

In [27]:
retention.head()
retention['cohort'] = retention['cohort'].astype(str)
retention['date_ym'] = retention['date_ym'].astype(str)

In [28]:
retention_final = pd.pivot_table(data=retention, index='cohort', columns='cohort_size_month', values='retention_rate')

In [29]:
fig = px.imshow(retention_final, text_auto='.2%',color_continuous_scale='Reds')
fig.show()

In [30]:
# 리텐션 커브
retention_curve = retention.groupby('cohort_size_month')[['retention_rate']].mean().reset_index()
retention_curve

,cohort_size_month,retention_rate
0,0,1.000000
1,1,0.391502
2,2,0.423605
3,3,0.434391
4,4,0.426799
5,5,0.424012
6,6,0.416228
7,7,0.411273
8,8,0.420212
9,9,0.432657


In [31]:
fig = px.line(data_frame=retention_curve, x='cohort_size_month', y='retention_rate',title='retention_curve')
fig.update_yaxes(tickformat='.2%')
fig.show()

rfm 분석  

In [32]:
# rm 계산
today_date = max(df['date_ymd'])

rfm = df.groupby('CustomerID').agg({'InvoiceDate' : lambda x : (today_date - x.max()).days,
                                     'amount' : lambda x: x.sum()   })
rfm.columns = ['recency','monetary']
rfm.head()

,recency,monetary
CustomerID,,
12346,324,77183.60
12347,1,4310.00
12348,74,1797.24
12349,17,1757.55
12350,309,334.40


In [34]:
pd.qcut(rfm['recency'],5,labels=[5,4,3,2,1])

CustomerID
12346    1
12347    5
12348    2
12349    4
12350    1
        ..
18280    1
18281    1
18282    5
18283    5
18287    3
Name: recency, Length: 4339, dtype: category
Categories (5, int64): [5 < 4 < 3 < 2 < 1]

In [36]:
rfm['recency_score'] = pd.qcut(rfm['recency'],3,labels=[3,2,1])
rfm['monetary_score'] = pd.qcut(rfm['monetary'],3,labels=[1,2,3])
rfm['rm_score'] = rfm['recency_score'].astype(str) + rfm['monetary_score'].astype(str)
rfm.reset_index(inplace=True)
rfm

,CustomerID,recency,monetary,recency_score,monetary_score,rm_score
0,12346,324,77183.60,1,3,13
1,12347,1,4310.00,3,3,33
2,12348,74,1797.24,2,3,23
3,12349,17,1757.55,3,3,33
4,12350,309,334.40,1,1,11
...,...,...,...,...,...,...
4334,18280,276,180.60,1,1,11
4335,18281,179,80.82,1,1,11
4336,18282,6,178.05,3,1,31
4337,18283,2,2094.88,3,3,33


In [37]:
rm_score = rfm.groupby('rm_score')[['CustomerID']].nunique().reset_index().rename({'CustomerID' : 'customer_count'},axis=1)

In [38]:
rm_score

,rm_score,customer_count
0,11,790
1,12,481
2,13,170
3,21,407
4,22,546
5,23,457
6,31,250
7,32,419
8,33,819


In [39]:
def categorize_customer(score) :
  if score == '33':
    return '최우수'  # 최신성, 구매 모두 상당히 높음
  elif score in ['32','23','22'] :
    return '우수'  # 최신성 구매 모두 높음
  elif score == '11' :
    return '휴면'  # 최신성 구매 모두 낮음
  elif score in ['12','13'] :
    return '이탈 방지'  # 구매는 높으나 최신성은 낮음
  elif score in ['31','21'] :
    return '구매 유도'  # 최신성은 높으나 구매는 낮음

In [40]:
rm_score['category'] = rm_score['rm_score'].apply(categorize_customer)

In [41]:
fig = px.treemap(data_frame=rm_score, path=['category'], values='customer_count')
fig.show()